In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
import json
import random
from transformers import AutoTokenizer
from datasets import Dataset

# Load intent data
with open("/content/Intent.json", "r") as file:
    data = json.load(file)

# Prepare conversation pairs
conversations = []
for intent in data["intents"]:
    for text in intent["text"]:
        response = random.choice(intent["responses"])  # Pick a random response
        conversations.append(f"User: {text}\nBot: {response}")

# Load GPT-2 tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Fix: Set padding token explicitly
tokenizer.pad_token = tokenizer.eos_token

# Tokenize conversation data
tokenized_data = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")

# Convert tokenized data into a Hugging Face Dataset
dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
})

# Print a sample to verify everything works
print(dataset[0])

# Now you can train your model with this dataset


{'input_ids': [12982, 25, 23748, 198, 20630, 25, 644, 460, 1312, 466, 329, 345, 30, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 

In [14]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [12]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00


In [21]:
import torch
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenized_data = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")

dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
})

training_args = TrainingArguments(
    output_dir="./chatbot_model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

trainer.train()


Step,Training Loss


TrainOutput(global_step=174, training_loss=2.3790793583310883, metrics={'train_runtime': 1053.3848, 'train_samples_per_second': 0.33, 'train_steps_per_second': 0.165, 'total_flos': 24508376064000.0, 'train_loss': 2.3790793583310883, 'epoch': 1.0})

In [ ]:
def chatbot_response(user_input):
    input_text = f"User: {user_input}\nBot:"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

# Test chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        break
    print("Bot:", chatbot_response(user_input))


You: What medicines can I buy to help me with my common cold?
Bot:  Common cold medicines include cold medicines, medicines that treat cold, and medicines that help with colds. Common cold medicines include medicines that treat colds, and medicines
You: fever prevention
Bot:  fever prevention involves regular vaccinations, regular exercise, and regular medical check-ups. Common cold-related diseases include colds, arthritis, and allergies. Common colds include pneumonia, pneumonia, and colds associated with
You: diabetes prevention
Bot:  Diabetes prevention includes regular exercise, regular exercise, and avoiding medications. Diabetes prevention includes medications, limiting sugar intake, and avoiding processed foods. Diabetes prevention includes regular blood tests, regular exercise, and avoiding processed foods. Diabetes


In [24]:
import shutil
from google.colab import files

# Replace 'chatbot_model' with your folder name
shutil.make_archive("chatbot_model", 'zip', "chatbot_model")

# Download the zip file
files.download("chatbot_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!cp -r chatbot_model /content/drive/MyDrive/